# Deep Agent with Runloop Backend

This notebook demonstrates how to create a deep agent that operates on files in a Runloop devbox.

## Setup: Import dependencies

In [1]:
import os
from runloop_api_client import Runloop
from deepagents.graph import create_deep_agent
from deepagents.integrations.runloop import RunloopProvider

## Option 1: Using RunloopProvider (Recommended)

The `RunloopProvider` will automatically create a devbox when the agent is initialized.

In [2]:
# Create the Runloop provider with optional devbox creation parameters
provider = RunloopProvider(
    api_key=os.environ.get("RUNLOOP_API_KEY"),
    create_params={
        # Optional: Add code mounts, environment variables, etc.
        # "code_mounts": [
        #     {
        #         "repo_name": "your-repo",
        #         "repo_owner": "your-username",
        #         "token": os.environ.get("GITHUB_TOKEN"),
        #     }
        # ],
        # "environment_variables": {"MY_VAR": "value"},
    }
)

# The None Here doesn't make much sense
backend = provider.backend_factory(None)

# Create the deep agent with the Runloop backend
agent = create_deep_agent(
    backend=backend,
    system_prompt="You are a helpful coding assistant that operates on files in a remote devbox.",
)

print("Agent created with Runloop backend!")

Agent created with Runloop backend!


In [4]:
agent.invoke({
    "messages": [
        { 
            "role": "user",
            "content": "could you list the files you have access to (not using the shell)"
        }
    ]}
            )

{'messages': [HumanMessage(content='could you list the files you have access to (not using the shell)', additional_kwargs={}, response_metadata={}, id='229d5971-55b2-44d2-9533-84a661ea4da4'),
  AIMessage(content=[{'text': "I'll list the files in the root directory for you.", 'type': 'text'}, {'id': 'toolu_01Ugvitaew9uS1ozKmrQari3', 'input': {'path': '/'}, 'name': 'ls', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_013MoxLmzkUc6SVWxUeAD7aa', 'model': 'claude-sonnet-4-5-20250929', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 5776}, 'cache_creation_input_tokens': 5776, 'cache_read_input_tokens': 0, 'input_tokens': 3, 'output_tokens': 64, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-5-20250929', 'model_provider': 'anthropic'}, id='lc_run--9b436cd0-b482-4211-ad71-6774481babed-0', tool_calls=[{'name': 'ls', 'args': {'path': '/'}, '

## With GitHub code mounts

Create an agent that operates on a cloned GitHub repository.

In [3]:
# Create provider with GitHub code mount
provider_with_repo = RunloopProvider(
    api_key=os.environ.get("RUNLOOP_API_KEY"),
    create_params={
        "code_mounts": [
            {
                "repo_name": "personal-site-generator",  # Replace with your repo
                "repo_owner": "eyurtsev",  # Replace with your username
                "token": os.environ.get("GITHUB_TOKEN"),
            }
        ],
    }
)

# The None Here doesn't make much sense
backend = provider.backend_factory(None)

# Create agent
agent = create_deep_agent(
    backend=backend,
    system_prompt="You are a helpful coding assistant. The repository is mounted in the devbox.",
)

print("Agent created with GitHub repository mounted!")

Agent created with GitHub repository mounted!


## Running the agent

Now you can interact with your agent and it will operate on files in the Runloop devbox.

In [4]:
import uuid

In [5]:
config = {
    "configurable": {
        "thread_id": str(uuid.uuid4())
    }
}

In [ ]:
# Example: Ask the agent to create a file
result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "list the mounted git repo'"
    }]
}, config=config)

# Print the agent's response
print(result["messages"][-1].content)